In [92]:
import json
import base64
import random
import hashlib
import requests
import numpy as np
from PIL import Image
from datetime import datetime
from cryptography.fernet import Fernet

import reedsolo

def generate_key():
    return Fernet.generate_key()

def generate_seed(key):
    # Convert key to bytes if not already
    if not isinstance(key, (bytes, bytearray)):
        key = str(key).encode()
    return int(hashlib.sha256(key).hexdigest(), 16) % (2**32)

def np_seed(key):
    return np.random.seed(generate_seed(key))

# Encrypt a message using symmetric encryption
def encrypt_message(message, key):
    cipher = Fernet(key)
    return cipher.encrypt(message.encode())

def decrypt_message(ciphertext, key):
    cipher = Fernet(key)
    return cipher.decrypt(ciphertext).decode()

def get_current_location():
    response = requests.get('https://ipinfo.io/json')
    data = response.json()
    city = data.get('city', '')
    region = data.get('region', '')
    country = data.get('country', '')
    loc = data.get('loc', '')  # latitude,longitude
    return {
        'city': city,
        'region': region,
        'country': country,
        'coordinates': loc
    }

## Spread Spectrum Encoder

In [93]:
"""
Encodes a message into an image using spread spectrum technique.

This function embeds a given message into an image by converting the message
into a bit sequence and spreading it across the image's pixel data using a
pseudorandom noise sequence generated from a key. The resulting watermarked
image is saved to the specified output path.

Parameters:
    image_path (str): Path to the input image file.
    message (str): The message to encode into the image.
    key (int): Seed for generating the pseudorandom noise sequence.
    output_path (str): Path to save the watermarked output image.

Raises:
    ValueError: If the image is too small to encode the message.
"""
def ss_encode(image_path, secret_message, key, output_path, nsym=32, alpha=0.1):
    # Encrypt and encode the message
    encrypted_message = encrypt_message(secret_message, key)
    encoded_message = base64.urlsafe_b64encode(encrypted_message)
    # Reed-Solomon encode (add nsym bytes of redundancy)
    rs = reedsolo.RSCodec(nsym)
    rs_encoded = rs.encode(encoded_message)
    message_bits = ''.join(format(byte, '08b') for byte in rs_encoded)
    message_length_bytes = len(rs_encoded)

    # Open image and convert to RGB
    image = Image.open(image_path).convert('RGB')
    image_data = np.array(image)
    watermarked_channels = []

    for channel in range(3):
        channel_data = image_data[..., channel].astype(np.float32)
        flat_channel = channel_data.flatten()

        if channel == 0:
            np_seed(key)
            pn_sequence = np.random.choice([-1, 1], size=flat_channel.shape)
            for i, bit in enumerate(message_bits):
                if i < len(flat_channel):
                    flat_channel[i] += alpha * pn_sequence[i] * (1 if bit == '1' else -1)
        watermarked_channel = flat_channel.reshape(channel_data.shape)
        watermarked_channel = np.clip(watermarked_channel, 0, 255).astype(np.uint8)
        watermarked_channels.append(watermarked_channel)

    watermarked_image = np.stack(watermarked_channels, axis=-1)
    Image.fromarray(watermarked_image).save(output_path)

    print("Embedding bits:", message_bits[:64])
    print("Total bits embedded:", len(message_bits))

    return message_length_bytes, nsym

## Spread Spectrum Decoder

In [94]:
"""
Decodes a message from an image using the spread spectrum technique.

This function extracts an embedded message from an image by correlating the
image's pixel data with a pseudorandom noise sequence generated from a key.
The message is reconstructed from the correlated bit sequence.

Parameters:
    image_path (str): Path to the input image file.
    message_length (int): Length of the message to decode.
    key (int): Seed for generating the pseudorandom noise sequence.

Returns:
    str: The decoded message from the image.
"""
def ss_decode(image_path, key, message_length, nsym=32, alpha=0.1):
    import reedsolo
    image = Image.open(image_path).convert('RGB')
    image_data = np.array(image)
    channel_data = image_data[..., 0].astype(np.float32)
    flat_channel = channel_data.flatten()

    np_seed(key)
    pn_sequence = np.random.choice([-1, 1], size=flat_channel.shape)

    bits = []
    for i in range(message_length * 8):
        if i < len(flat_channel):
            correlation = flat_channel[i] * pn_sequence[i]
            bits.append(1 if correlation > 0 else 0)

    print("Extracted bits:", bits[:64])
    print("Total bits extracted:", len(bits))

    # Convert bits to bytes
    bytes_out = bytearray()
    for i in range(0, len(bits), 8):
        byte = bits[i:i+8]
        if len(byte) < 8:
            break
        bytes_out.append(int(''.join(map(str, byte)), 2))

    try:
        rs = reedsolo.RSCodec(nsym)
        decoded_bytes = rs.decode(bytes_out)
        # Remove RS parity bytes
        decoded_bytes = decoded_bytes[:-(nsym)] if nsym > 0 else decoded_bytes
        # Now decode base64 and decrypt
        encrypted_message = base64.urlsafe_b64decode(decoded_bytes)
        message = decrypt_message(encrypted_message, key)
        return message
    except Exception as e:
        return f"Decoding failed: {e}"

## Usage example

In [95]:
inputPath = "assets/images/raw.png"
outputPath = "assets/images/ss-encoded.png"

secret_data = {
    "message": "Very Secret Message",
    "date": datetime.now().isoformat(),
    "location": get_current_location()
}

secret_message = "hello" #json.dumps(secret_data, indent=4)

key = generate_key()
print("Generated encryption key (store this securely):")
print(key.decode())

# To encode:
encoded_len, nsym = ss_encode(inputPath, secret_message, key, outputPath, 128, 32)

# To decode:
message = ss_decode(outputPath, key, encoded_len, nsym, 32)
print("\nDecoded message:")
print(message)

Generated encryption key (store this securely):
dWuz75E-82bDyOhl93BQ14uoUZfuHy6vDOypGuwj2-s=
Embedding bits: 0101101000110000010001100100001001010001010101010100011001000010
Total bits embedded: 3136
Extracted bits: [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Total bits extracted: 3136

Decoded message:
Decoding failed: Too many (or few) errors found by Chien Search for the errata locator polynomial!
